In [ ]:
import pandas as pd
from openprompt.data_utils import InputExample

# ==============================
# Load STS-B labeled dataset
# ==============================
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\STS-B\Final_STSB_train_labeled.csv"
)

print("Columns:", df.columns.tolist())

# ==============================
# Label Mapping (5-way classification)
# ==============================
label_map = {
    "unrelated": 0,
    "distant": 1,
    "similar": 2,
    "equivalent": 3,
    "identical": 4
}

classes = list(label_map.keys())

guid = 0
examples = []
class_stats = {}

# ==============================
# Take FIRST 16 examples per label
# ==============================
for label_name in classes:
    class_df = df[df["score_to_labels"] == label_name].head(16)
    class_stats[label_name] = len(class_df)

    for _, row in class_df.iterrows():
        examples.append(
            InputExample(
                guid=guid,
                text_a=str(row["sentence1"]).strip(),
                text_b=str(row["sentence2"]).strip(),
                label=label_map[label_name]
            )
        )
        guid += 1

# ==============================
# Print InputExamples (exact format)
# ==============================
print("\n### InputExamples ###\n")
for ex in examples:
    print(
        "InputExample(\n"
        f"    guid={ex.guid},\n"
        f"    text_a=\"{ex.text_a}\",\n"
        f"    text_b=\"{ex.text_b}\",\n"
        f"    label={ex.label}\n"
        "),"
    )

# ==============================
# Metadata
# ==============================
print("\n### Classes ###")
print(f"classes = {classes}")

print("\n### Label Map ###")
print(f"label_map = {label_map}")

print("\n### Samples per Class ###")
for cls, count in class_stats.items():
    print(f"{cls}: {count}")

print(f"\nTotal InputExamples generated: {guid}")


In [ ]:
templates = [

    ("P1", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} آپس میں {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),

    ("P2", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے معنی {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),

    ("P3", ManualTemplate(
        text='{"placeholder":"text_a"} کا {"placeholder":"text_b"} سے تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P4", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} معنی کے لحاظ سے {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),

    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} ایک دوسرے سے {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),

    ("P6", ManualTemplate(
        text='{"placeholder":"text_a"} کو {"placeholder":"text_b"} سے ملایا جائے تو نتیجہ {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P7", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا مفہوم {"mask"} بنتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P8", ManualTemplate(
        text='{"placeholder":"text_a"} کے مقابلے میں {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} میں معنی کا رشتہ {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} ایک جیسے ہونے کے لحاظ سے {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),
]


In [ ]:
verbalizer = {
    "unrelated": ["غیرمتعلق", "الگ"],
    "distant": ["کم", "دور"],
    "similar": ["ملتے", "مشابہ"],
    "equivalent": ["تقریباً", "برابر"],
    "identical": ["بالکل", "ایک"],
}


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [
    
        InputExample(
            guid=0,
            text_a="ایک آدمی سگریٹ پی رہا ہے ۔",
            text_b="ایک آدمی سکیٹنگ کر رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=1,
            text_a="ایک عورت لکھ رہی ہے ۔",
            text_b="ایک عورت تیر رہی ہے ۔",
            label=0
        ),
        InputExample(
            guid=2,
            text_a="ایک آدمی ناچ رہا ہے ۔",
            text_b="ایک آدمی بات کر رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=3,
            text_a="ایک آدمی مچھلی پکڑ رہا ہے ۔",
            text_b="ایک آدمی ورزش کر رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=4,
            text_a="ایک آدمی لہرا رہا ہے ۔",
            text_b="ایک آدمی بات کر رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=5,
            text_a="دو لڑکے گاڑی چلا رہے ہیں ۔",
            text_b="دو بے رقص کر رہے ہیں ۔",
            label=0
        ),
        InputExample(
            guid=6,
            text_a="ایک عورت ناچ رہی ہے ۔",
            text_b="ایک آدمی کھا رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=7,
            text_a="ایک بلی پیانو بجا رہی ہے ۔",
            text_b="ایک آدمی گٹار بجا رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=8,
            text_a="ایک عورت ناچ رہی ہے ۔",
            text_b="ایک آدمی بات کر رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=9,
            text_a="ایک آدمی کشتی چلا رہا ہے ۔",
            text_b="ایک آدمی ٹماٹر کاٹ رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=10,
            text_a="ایک آدمی کینو کو پیڈل کر رہا ہے ۔",
            text_b="ایک آدمی ہارپ بجا رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=11,
            text_a="ایک آدمی تندور میں پین ڈال رہا ہے ۔",
            text_b="ایک عورت پیاز کاٹ رہی ہے ۔",
            label=0
        ),
        InputExample(
            guid=12,
            text_a="ایک عورت میز پر بیٹھی ہے ۔",
            text_b="ایک عورت گدھے پر سوار ہے ۔",
            label=0
        ),
        InputExample(
            guid=13,
            text_a="ایک آدمی کاغذ کو تہہ کر رہا ہے ۔",
            text_b="ایک عورت کالی مرچ کاٹ رہی ہے ۔",
            label=0
        ),
        InputExample(
            guid=14,
            text_a="ایک آدمی گٹار بجا رہا ہے ۔",
            text_b="ایک آدمی شیشے کے دروازے کی صفائی کر رہا ہے ۔",
            label=0
        ),
        InputExample(
            guid=15,
            text_a="ایک عورت وزن کی مشقیں کر رہی ہے ۔",
            text_b="ایک عورت اپنے بال بنا رہی ہے ۔",
            label=0
        ),
        InputExample(
            guid=16,
            text_a="آدمی پیانو بجا رہا ہے ۔",
            text_b="آدمی گٹار بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=17,
            text_a="ایک آدمی سڑک پر دوڑ رہا ہے ۔",
            text_b="ایک پانڈا کتا سڑک پر دوڑ رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=18,
            text_a="شیر کا بچہ گیند سے کھیل رہا ہے ۔",
            text_b="ایک بچہ گڑیا کے ساتھ کھیل رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=19,
            text_a="ایک شخص ٹماٹر کاٹ رہا ہے ۔",
            text_b="ایک شخص کچھ گوشت کاٹ رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=20,
            text_a="ایک آدمی پیانو بجا رہا ہے ۔",
            text_b="ایک عورت وائلن بجا رہی ہے ۔",
            label=1
        ),
        InputExample(
            guid=21,
            text_a="ایک عورت بانسری بجا رہی ہے ۔",
            text_b="ایک آدمی گٹار بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=22,
            text_a="ایک آدمی گٹار بجا رہا ہے ۔",
            text_b="ایک آدمی ڈھول بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=23,
            text_a="ایک بلی کلیدی تختہ بجا رہی ہے ۔",
            text_b="ایک آدمی دو کی بورڈ بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=24,
            text_a="ایک آدمی موٹر سائیکل چلا رہا ہے ۔",
            text_b="ایک آدمی گھوڑے پر سوار ہے ۔",
            label=1
        ),
        InputExample(
            guid=25,
            text_a="ایک آدمی موٹر سائیکل چلا رہا ہے ۔",
            text_b="ایک آدمی گھوڑے پر سوار ہے ۔",
            label=1
        ),
        InputExample(
            guid=26,
            text_a="ایک آدمی پیانو بجا رہا ہے ۔",
            text_b="ایک آدمی صور بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=27,
            text_a="ایک عورت ٹماٹر کاٹ رہی ہے ۔",
            text_b="ایک آدمی پیاز کاٹ رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=28,
            text_a="ایک مرد اور عورت بات کر رہے ہیں ۔",
            text_b="ایک مرد اور عورت کھا رہے ہیں ۔",
            label=1
        ),
        InputExample(
            guid=29,
            text_a="آدمی گٹار بجا رہا ہے ۔",
            text_b="ایک آدمی بانسری بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=30,
            text_a="ایک آدمی پیانو بجا رہا ہے ۔",
            text_b="آدمی وائلن بجا رہا ہے ۔",
            label=1
        ),
        InputExample(
            guid=31,
            text_a="ایک آدمی سڑک پر دوڑ رہا ہے ۔",
            text_b="ایک کار سڑک پر چل رہی ہے ۔",
            label=1
        ),
        InputExample(
            guid=32,
            text_a="تین آدمی شطرنج کھیل رہے ہیں ۔",
            text_b="دو آدمی شطرنج کھیل رہے ہیں ۔",
            label=2
        ),
        InputExample(
            guid=33,
            text_a="ایک آدمی گٹار بجا رہا ہے اور گا رہا ہے ۔",
            text_b="ایک عورت صوتی گٹار بجا رہی ہے اور گا رہی ہے ۔",
            label=2
        ),
        InputExample(
            guid=34,
            text_a="ایک آدمی گٹار بجا رہا ہے ۔",
            text_b="ایک لڑکی گٹار بجا رہی ہے ۔",
            label=2
        ),
        InputExample(
            guid=35,
            text_a="ایک آدمی ایک بن کاٹ رہا ہے ۔",
            text_b="ایک آدمی پیاز کاٹ رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=36,
            text_a="ایک عورت پیاز کاٹ رہی ہے ۔",
            text_b="ایک آدمی پیاز کاٹ رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=37,
            text_a="ایک آدمی آلو کاٹ رہا ہے ۔",
            text_b="ایک آدمی گاجر کاٹ رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=38,
            text_a="ایک آدمی گٹار بجا رہا ہے اور گا رہا ہے ۔",
            text_b="ایک آدمی گٹار بجا رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=39,
            text_a="شیر کا بچہ آواز اٹھا رہا ہے ۔",
            text_b="ایک شیر ادھر ادھر گھوم رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=40,
            text_a="ایک شخص پیاز کاٹ رہا ہے ۔",
            text_b="ایک شخص پیاز چھیل رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=41,
            text_a="ایک عورت آلو چھیل رہی ہے ۔",
            text_b="ایک عورت سیب چھیل رہی ہے ۔",
            label=2
        ),
        InputExample(
            guid=42,
            text_a="ایک شخص پیاز چھیل رہا ہے ۔",
            text_b="ایک شخص بینگن چھیل رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=43,
            text_a="ایک عورت گٹار بجا رہی ہے ۔",
            text_b="ایک آدمی گٹار بجاتا ہے ۔",
            label=2
        ),
        InputExample(
            guid=44,
            text_a="ایک آدمی پانی کے اندر تیرتا ہے ۔",
            text_b="ایک عورت پانی کے اندر تیر رہی ہے ۔",
            label=2
        ),
        InputExample(
            guid=45,
            text_a="مرد کرکٹ کھیل رہے ہیں ۔",
            text_b="مرد باسکٹ بال کھیل رہے ہیں ۔",
            label=2
        ),
        InputExample(
            guid=46,
            text_a="ایک آدمی گھوڑے پر سوار ہے ۔",
            text_b="ایک لڑکی گھوڑے پر سوار ہے ۔",
            label=2
        ),
        InputExample(
            guid=47,
            text_a="ایک آدمی سائیکل چلا رہا ہے ۔",
            text_b="ایک بندر سائیکل چلا رہا ہے ۔",
            label=2
        ),
        InputExample(
            guid=48,
            text_a="ایک آدمی بڑی بانسری بجا رہا ہے ۔",
            text_b="ایک آدمی بانسری بجا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=49,
            text_a="ایک آدمی پیزا پر کٹا ہوا پنیر پھیلا رہا ہے ۔",
            text_b="ایک آدمی بغیر پکے ہوئے پیزا پر کٹا ہوا پنیر پھیلا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=50,
            text_a="ایک آدمی بانسری بجا رہا ہے ۔",
            text_b="ایک آدمی بانس کی بانسری بجا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=51,
            text_a="ایک کتا اپنی پیٹھ سے بیکن اتارنے کی کوشش کر رہا ہے ۔",
            text_b="ایک کتا اپنی پیٹھ پر لگے بیکن کو کھانے کی کوشش کر رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=52,
            text_a="ایک بلی بچے کے چہرے پر رگڑ رہی ہے ۔",
            text_b="ایک بلی بچے کے خلاف رگڑ رہی ہے ۔",
            label=3
        ),
        InputExample(
            guid=53,
            text_a="ایک آدمی برتن میں تیل ڈالتا ہے ۔",
            text_b="ایک آدمی برتن میں شراب ڈالتا ہے ۔",
            label=3
        ),
        InputExample(
            guid=54,
            text_a="ایک عورت کچھ کھا رہی ہے ۔",
            text_b="ایک عورت گوشت کھا رہی ہے ۔",
            label=3
        ),
        InputExample(
            guid=55,
            text_a="ایک عورت دوسری عورتوں کے ساتھ ناچ رہی ہے اور گا رہی ہے ۔",
            text_b="ایک عورت بارش میں ناچ رہی ہے اور گا رہی ہے ۔",
            label=3
        ),
        InputExample(
            guid=56,
            text_a="شیر لوگوں کے ساتھ کھیل رہا ہے ۔",
            text_b="شیر دو آدمیوں کے ساتھ کھیل رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=57,
            text_a="کوئی مجسمہ تراش رہا ہے ۔",
            text_b="ایک آدمی مجسمہ تراش رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=58,
            text_a="ایک عورت الیکٹرک گٹار بجا رہی ہے ۔",
            text_b="ایک عورت گٹار بجا رہی ہے ۔",
            label=3
        ),
        InputExample(
            guid=59,
            text_a="ایک بچہ گٹار بجا رہا ہے ۔",
            text_b="ایک لڑکا گٹار بجا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=60,
            text_a="ایک لڑکا گٹار بجا رہا ہے ۔",
            text_b="ایک آدمی گٹار بجا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=61,
            text_a="ایک آدمی گٹار بجا رہا ہے ۔",
            text_b="ایک لڑکا گٹار بجا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=62,
            text_a="ایک آدمی گٹار بجا رہا ہے ۔",
            text_b="ایک آدمی برقی گٹار بجا رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=63,
            text_a="ایک عورت پیاز کاٹ رہی ہے ۔",
            text_b="ایک آدمی پیاز کاٹ رہا ہے ۔",
            label=3
        ),
        InputExample(
            guid=64,
            text_a="ایک طیارہ اڑان بھر رہا ہے ۔",
            text_b="ایک ہوائی جہاز اڑان بھر رہا ہے ۔",
            label=4
        ),
        InputExample(
            guid=65,
            text_a="ایک آدمی سیلو کھیل رہا ہے ۔",
            text_b="بیٹھا ہوا آدمی سیلو کھیل رہا ہے ۔",
            label=4
        ),
        InputExample(
            guid=66,
            text_a="کچھ آدمی لڑ رہے ہیں ۔",
            text_b="دو آدمی لڑ رہے ہیں ۔",
            label=4
        ),
        InputExample(
            guid=67,
            text_a="ایک شخص ایک بلی کو چھت پر پھینک رہا ہے ۔",
            text_b="ایک شخص ایک بلی کو چھت پر پھینک دیتا ہے ۔",
            label=4
        ),
        InputExample(
            guid=68,
            text_a="آدمی نے دوسرے آدمی کو چھڑی سے مارا ۔",
            text_b="اس شخص نے دوسرے آدمی کو چھڑی سے مارا ۔",
            label=4
        ),
        InputExample(
            guid=69,
            text_a="ایک عورت اٹھاتی ہے اور بچے کے کینگرو کو پکڑتی ہے ۔",
            text_b="ایک عورت اٹھاتی ہے اور اپنے بازوؤں میں کینگرو کے بچے کو پکڑتی ہے ۔",
            label=4
        ),
        InputExample(
            guid=70,
            text_a="ایک شخص کاغذ کا ایک ٹکڑا تہہ کر رہا ہے ۔",
            text_b="کوئی کاغذ کا ٹکڑا جوڑ رہا ہے ۔",
            label=4
        ),
        InputExample(
            guid=71,
            text_a="قطبی ریچھ برف پر پھسل رہا ہے ۔",
            text_b="ایک قطبی ریچھ برف کے پار پھسل رہا ہے ۔",
            label=4
        ),
        InputExample(
            guid=72,
            text_a="آدمی گھوڑے پر سوار ہے ۔",
            text_b="ایک آدمی گھوڑے پر سوار ہے ۔",
            label=4
        ),
        InputExample(
            guid=73,
            text_a="ایک پانڈا ایک سلائیڈ سے نیچے پھسل رہا ہے ۔",
            text_b="ایک پانڈا ایک سلائیڈ سے نیچے پھسلتا ہے ۔",
            label=4
        ),
        InputExample(
            guid=74,
            text_a="ایک عورت آلو چھیلتی ہے ۔",
            text_b="ایک عورت آلو چھیل رہی ہے ۔",
            label=4
        ),
        InputExample(
            guid=75,
            text_a="لڑکا اپنی موٹر سائیکل سے گر گیا ۔",
            text_b="ایک لڑکا اپنی موٹر سائیکل سے گر جاتا ہے ۔",
            label=4
        ),
        InputExample(
            guid=76,
            text_a="عورت بانسری بجا رہی ہے ۔",
            text_b="ایک عورت بانسری بجا رہی ہے ۔",
            label=4
        ),
        InputExample(
            guid=77,
            text_a="خرگوش عقاب سے دوڑ رہا ہے ۔",
            text_b="ایک خرگوش عقاب سے دوڑ رہا ہے ۔",
            label=4
        ),
        InputExample(
            guid=78,
            text_a="عورت روٹی والے سور کے گوشت کے ٹکڑے کو تلی رہی ہے ۔",
            text_b="ایک عورت روٹی والا سور کا گوشت پکا رہی ہے ۔",
            label=4
        ),
        InputExample(
            guid=79,
            text_a="ایک لڑکی پتنگ اڑا رہی ہے ۔",
            text_b="ایک لڑکی دوڑتی ہوئی پتنگ اڑا رہی ہے ۔",
            label=4
        ),
        
    
]


In [4]:
### Classes ###
classes = ['unrelated', 'distant', 'similar', 'equivalent', 'identical']

### Label Map ###
label_map = {'unrelated': 0, 'distant': 1, 'similar': 2, 'equivalent': 3, 'identical': 4}



# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

In [5]:
# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} آپس میں {"mask"} ہیں۔',
    tokenizer=tokenizer,
)











verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "unrelated": ["غیرمتعلق", "الگ"],
        "distant": ["کم", "دور"],
        "similar": ["ملتے", "مشابہ"],
        "equivalent": ["تقریباً", "برابر"],
        "identical": ["بالکل", "ایک"]
    },
    tokenizer=tokenizer,
)

In [6]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # Even though shuffled, reproducibility is preserved by seeding
)

# # ==============================
# # Fine-Tuning the Prompt Model
# # ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(20):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

tokenizing: 80it [00:00, 1792.57it/s]


Epoch 1 Loss: 38.3621
Epoch 2 Loss: 32.0704
Epoch 3 Loss: 28.9695
Epoch 4 Loss: 24.2436
Epoch 5 Loss: 22.2553
Epoch 6 Loss: 14.1873
Epoch 7 Loss: 13.9126
Epoch 8 Loss: 10.3821
Epoch 9 Loss: 7.1365
Epoch 10 Loss: 3.7427
Epoch 11 Loss: 8.3980
Epoch 12 Loss: 7.1995
Epoch 13 Loss: 1.6894
Epoch 14 Loss: 2.7074
Epoch 15 Loss: 1.4500
Epoch 16 Loss: 3.6499
Epoch 17 Loss: 4.8538
Epoch 18 Loss: 5.3206
Epoch 19 Loss: 2.7530
Epoch 20 Loss: 1.8344


In [7]:
# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\STS-B\Final_dev_labeled.csv"
)

# Make InputExamples
eval_dataset = [
    InputExample(
        guid=i,
        text_a=row['sentence1'],
        text_b=row['sentence2'],
        label=label_map[row['score_to_labels']]
    )
    for i, row in df.iterrows()
]

# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate for XNLI
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 STS_B Urdu Dev Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

tokenizing: 1500it [00:00, 1854.43it/s]



📊 STS_B Urdu Dev Classification Report:
              precision    recall  f1-score   support

   unrelated     0.4928    0.3953    0.4387       344
     distant     0.2326    0.3543    0.2808       254
     similar     0.2286    0.0293    0.0519       273
  equivalent     0.3536    0.4630    0.4009       365
   identical     0.3951    0.4848    0.4354       264

    accuracy                         0.3540      1500
   macro avg     0.3405    0.3454    0.3216      1500
weighted avg     0.3495    0.3540    0.3318      1500



In [ ]:
roberta

📊 STS_B Urdu Dev Classification Report:
              precision    recall  f1-score   support

   unrelated     0.5424    0.3721    0.4414       344
     distant     0.3025    0.1417    0.1930       254
     similar     0.4091    0.0330    0.0610       273
  equivalent     0.2904    0.5808    0.3872       365
   identical     0.2824    0.4205    0.3379       264

    accuracy                         0.3307      1500
   macro avg     0.3654    0.3096    0.2841      1500
weighted avg     0.3704    0.3307    0.2987      1500

In [ ]:
mbert


📊 STS_B Urdu Dev Classification Report:
              precision    recall  f1-score   support

   unrelated     0.4928    0.3953    0.4387       344
     distant     0.2326    0.3543    0.2808       254
     similar     0.2286    0.0293    0.0519       273
  equivalent     0.3536    0.4630    0.4009       365
   identical     0.3951    0.4848    0.4354       264

    accuracy                         0.3540      1500
   macro avg     0.3405    0.3454    0.3216      1500
weighted avg     0.3495    0.3540    0.3318      1500